In [2]:
import csv
from pathlib import Path
import plotly.graph_objects as go


# Map csv data to variables
path = Path("~/.finances").expanduser() / "historical_transactions" / "2024_sankey_data.csv"
data = {}
with open(path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data[row['item']] = int(row['amount'])


wages_a = data['wages_a'] # Austin
wages_b = data['wages_b'] # Becca
savings_int = data['savings_int']
rh_div = data['rh_div']
rh_gains = data['rh_gains']
rh_costbasis = data['rh_costbasis']
cc_stmt_credits = data['cc_stmt_credits'] # Statement credits for Amazon, AmEx, USBank only

anchor = data['anchor']
tax_ret_2023 = data['tax_ret_2023'] # $30 cost to file
wedding_gift = data['wedding_gift']

emp_match_a = data['emp_match_a']
emp_match_b = data['emp_match_b']
hsa = data['hsa']
fers = data['fers']

rh_disc_tax_fed = (rh_div + rh_gains) * 0.24 + savings_int * 0.15
rh_disc_tax_state = (savings_int + rh_div + rh_gains) * 0.0637
disc_tax_fica = savings_int * (0.062 + 0.0145)
tax_fed = data['tax_fed_wages'] + rh_disc_tax_fed
tax_state = data['tax_state_wages'] + rh_disc_tax_state
tax_fica = data['tax_fica_wages'] + disc_tax_fica

ira_2023_max = 6500
roth_iras = ira_2023_max * 2
roth_tsp = data['roth_tsp']
roth_401k = data['roth_401k']

health_ins_a = data['health_ins_a']
health_ins_b = data['health_ins_b']
fegli = data['fegli']

net_a = data['net_a']
net_b = data['net_b']
net_rh_disc = savings_int + rh_div + rh_gains - (rh_disc_tax_fed + rh_disc_tax_state + disc_tax_fica)

# Housing:
rent = data['rent']
rent_ins = data['rent_ins']
utilities = data['utilities'] # PSEG, Internet, Phone
housing_and_utilities = rent + rent_ins + utilities

# Car (no loans):
car_ins = data['car_ins']
gas = data['gas']
tolls = data['tolls']
car_maint = data['car_maint'] # For one oil change and registration renewal
car_total_no_loans = car_ins + gas + tolls + car_maint

# Loans:
car_loan = data['car_loan']
car_loan_parents = data['car_loan_parents'] # Only 3 payments (1/1, 4/1, 7/1)
wedding_loan = data['wedding_loan'] # 4 payments (1/1, 4/1, 7/1, 10/1)
student_loans = data['student_loans']
loans_total = car_loan + car_loan_parents + wedding_loan + student_loans

# Health:
becca_gyms = data['becca_gyms'] # Assume same price as last year for Base annual membership?
austin_gym = data['austin_gym'] # Assuming monthly w/o switch to annual in Nov.
medical = data['medical'] # Medical/doctor/therapy/dental bills and CVS/Duane Reade/etc charges
health = becca_gyms + austin_gym + medical

# Beauty:
haircuts = data['haircuts']
cosmetics = data['cosmetics']
clothes = data['clothes']
beauty = haircuts + cosmetics + clothes

# Subscriptions and fees:
subscriptions = data['subscriptions'] # Headspace, Amazon, Amazon photos, apple, paramount+, adobe, spotify
cc_fees = data['cc_fees']
subscriptions_and_fees = subscriptions + cc_fees

# Food/groceries/supplies/staples:
groceries_and_home = data['groceries_and_home'] # Includes house supplies, home improvement, etc.
restaurants = data['restaurants']
food_and_supplies = groceries_and_home + restaurants

# Variable discretionary:
activities = data['activities'] # Concerts, bars, NYC activities, liquor store purchases, etc. (excludes travel)
travel = data['travel']
gifts = data['gifts'] # Gifts for anyone other than us two
other_discretionary = data['other_discretionary'] # New headphones, parking tickets, fridge magnets, etc.
variable_discretionary = activities + travel + gifts + other_discretionary

# Excess savings:
net_income = sum((
    net_a,
    net_b,
    net_rh_disc,
    anchor,
    tax_ret_2023,
    wedding_gift,
    rh_costbasis,
    cc_stmt_credits
))
excess_savings = net_income - sum((
    housing_and_utilities,
    car_total_no_loans,
    loans_total,
    health,
    beauty,
    subscriptions_and_fees,
    food_and_supplies,
    variable_discretionary
))
print(f"Excess Savings: ${int(excess_savings):,}")


# Check Gross Income inflows and outflows (should be equal)
gross_in = wages_a + wages_b + savings_int + rh_div + rh_gains
gross_out = sum((
    hsa,
    fers,
    roth_iras,
    roth_tsp,
    roth_401k,
    tax_fed,
    tax_state,
    tax_fica,
    health_ins_a,
    health_ins_b,
    fegli,
    net_a,
    net_b,
    net_rh_disc
))
print(f"Gross In: ${gross_in:,}", "|", f"Gross Out: ${int(gross_out):,}" )


# List of tuples in the form (source, value, target)
flows = [
    # Income Sources
    ("Wages (Austin)", wages_a, "Gross Income"),
    ("Wages (Becca)", wages_b, "Gross Income"),
    ("Savings Acc. Interest", savings_int, "Gross Income"),
    ("RH House Acc. Dividends", rh_div, "Gross Income"),
    ("RH House Acc. LT Cap. Gains", rh_gains, "Gross Income"),

    ("ANCHOR", anchor, "Other 'Income'"),
    ("2023 Tax Return", tax_ret_2023, "Other 'Income'"),
    ("Late Wedding Gift", wedding_gift, "Other 'Income'"),
    ("RH House Acc. Cost-Basis Withdrawal", rh_costbasis, "Other 'Income'"),
    ("CC Rewards Statement Credits", cc_stmt_credits, "Other 'Income'"),

    # Retirement
    ("Employer Match (TSP & 401k, Trad.)", emp_match_a + emp_match_b, "Pre-Tax Retirement"),
    ("Gross Income", hsa + fers, "Pre-Tax Retirement"),
    ("Gross Income", roth_iras + roth_tsp + roth_401k, "Post-Tax Retirement (Roth IRAs Included)"),

    ("Pre-Tax Retirement", emp_match_a, "TSP (Trad.)"),
    ("Pre-Tax Retirement", emp_match_b, "401k (Trad.)"),
    ("Pre-Tax Retirement", hsa, "HSA"),
    ("Pre-Tax Retirement", fers, "FERS (Pension)"),

    ("Post-Tax Retirement (Roth IRAs Included)", roth_iras, "Roth IRAs x2"),
    ("Post-Tax Retirement (Roth IRAs Included)", roth_tsp, "Roth TSP"),
    ("Post-Tax Retirement (Roth IRAs Included)", roth_401k, "Roth 401k"),

    ("TSP (Trad.)", emp_match_a, "Retirement (Total)"),
    ("401k (Trad.)", emp_match_b, "Retirement (Total)"),
    ("HSA", hsa, "Retirement (Total)"),
    ("FERS (Pension)", fers, "Retirement (Total)"),
    ("Roth IRAs x2", roth_iras, "Retirement (Total)"),
    ("Roth TSP", roth_tsp, "Retirement (Total)"),
    ("Roth 401k", roth_401k, "Retirement (Total)"),

    # Taxes
    ("Gross Income", tax_fed + tax_state + tax_fica, "Taxes"),
    ("Taxes", tax_fed, "Federal"),
    ("Taxes", tax_state, "State"),
    ("Taxes", tax_fica, "FICA"),

    # Insurance Premiums
    ("Gross Income", health_ins_a + health_ins_b + fegli, "Insurance Premiums"),
    ("Insurance Premiums", health_ins_b, "Med/Dent/Vis (Becca)"),
    ("Insurance Premiums", health_ins_a, "FEHB/FEDVIP (Austin)"),
    ("Insurance Premiums", fegli, "FEGLI"),

    # Net Income
    ("Gross Income", net_a + net_b + net_rh_disc, "Net Income (Roth IRAs Excluded)"),
    ("Other 'Income'", anchor + tax_ret_2023 + wedding_gift + rh_costbasis + cc_stmt_credits, "Net Income (Roth IRAs Excluded)"),

    ("Net Income (Roth IRAs Excluded)", housing_and_utilities, "Housing & Utilities"),
    ("Net Income (Roth IRAs Excluded)", car_total_no_loans, "Car (Loan Excluded)"),
    ("Net Income (Roth IRAs Excluded)", loans_total, "Loans"),
    ("Net Income (Roth IRAs Excluded)", health, "Health"),
    ("Net Income (Roth IRAs Excluded)", beauty, "Beauty"),
    ("Net Income (Roth IRAs Excluded)", subscriptions_and_fees, "Subscriptions & Fees"),
    ("Net Income (Roth IRAs Excluded)", food_and_supplies, "Food & Supplies"),
    ("Net Income (Roth IRAs Excluded)", variable_discretionary, "General Discretionary"),
    ("Net Income (Roth IRAs Excluded)", excess_savings, "Excess Savings"),

    ("Housing & Utilities", rent, "Rent"),
    ("Housing & Utilities", rent_ins, "Renter's Insurance"),
    ("Housing & Utilities", utilities, "Utilities"),

    ("Car (Loan Excluded)", car_total_no_loans, "Car Insurance"),
    ("Car (Loan Excluded)", gas + tolls + car_maint, "Gas, Tolls & Maintenance"),

    ("Loans", car_loan + car_loan_parents, "Car Loan"),
    ("Loans", wedding_loan, "Wedding Loan"),
    ("Loans", student_loans, "Student Loans"),

    ("Health", becca_gyms, "Becca Gym"),
    ("Health", austin_gym, "Austin Gym"),
    ("Health", medical, "Medical (Post-Tax)"),

    ("Beauty", haircuts, "Hair Care"),
    ("Beauty", cosmetics, "Cosmetics"),
    ("Beauty", clothes, "Clothes"),

    ("Subscriptions & Fees", subscriptions, "Subscriptions"),
    ("Subscriptions & Fees", cc_fees, "CC Fees"),

    ("Food & Supplies", groceries_and_home, "Groceries & Home"),
    ("Food & Supplies", restaurants, "Restaurants"),

    ("General Discretionary", activities, "Activities"),
    ("General Discretionary", travel, "Travel"),
    ("General Discretionary", gifts, "Gifts"),
    ("General Discretionary", other_discretionary, "Other Discretionary"),
]

# Extract unique labels (nodes) and map each label to its index
labels = list(set([item for src, _, tgt in flows for item in (src, tgt)]))
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Map source and target labels to indices using the dictionary
source_indices = [label_to_index[src] for src, _, tgt in flows]
target_indices = [label_to_index[tgt] for src, _, tgt in flows]
values = [value for _, value, _ in flows]

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=30,
        line=dict(color="black", width=1),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(
    title_text="Projected 2024 Income and Expenses",
    height=800,
    font_size=10
)
fig.show()


Excess Savings: $0
Gross In: $235,177 | Gross Out: $235,177
